# Avoiding overshoot: figures

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.kaleido.scope.default_width = None
pio.kaleido.scope.default_height = None

from utils import data, plotparams

In [ ]:
selection_wide = data.read_all("data/carbonbudget/")

In [ ]:
selection = data.to_long(selection_wide)

## Figure 1

In [ ]:
selection_fig1 = data.select(selection, cost_level='p50', tcre=data.DEFAULT_TCRE, endyear=2100, damage_i=2, perc_reversible=1, budget="600 GtCO2")

In [ ]:
import utils.fig1ab, utils.fig1c

In [ ]:
fig1ab = utils.fig1ab.create_fig(selection_fig1)
fig1ab

In [ ]:
fig1c = utils.fig1c.create_fig(selection_fig1, sdr=0.04)
fig1c

## Figure 2: sensitivity: total costs over time

In [ ]:
selection_fig2 = data.select(selection, cost_level='p50', tcre=data.DEFAULT_TCRE, endyear=2100, damage_i=[0, 2, -1], perc_reversible=1, budget="600 GtCO2")

In [ ]:
import utils.fig2

In [ ]:
subselect_costs = data.select(selection_fig2, damage_i=2, prtp=0.015)
subselect_costs[subselect_costs['year'].isin([2030, 2060, 2090])][['year', 'minlevel', 'total_costs_relative']]

In [ ]:
fig2 = utils.fig2.create_fig(selection_fig2, sdr=0.04)
fig2

In [ ]:
fig2_1344GtCO2 = utils.fig2.create_fig(data.select(selection, cost_level='p50', tcre=data.DEFAULT_TCRE, endyear=2100, damage_i=[0, 2, -1], perc_reversible=1, budget="1344 GtCO2"), sdr=0.04, with_arrows=False)
fig2_1344GtCO2

In [ ]:
# #### Until 2150:
# (
#     utils.fig2.create_fig(
#         data.select(selection, cost_level='p50', tcre=data.DEFAULT_TCRE, endyear=2150, damage_i=[0, 2, -1], perc_reversible=1, budget="600 GtCO2"),
#         sdr=0.04
#     )
#     .update_xaxes(col=1, range=[2020, 2150])
#     .update_xaxes(col=3, range=[2020, 2150])
#     .update_xaxes(col=5, range=[2020, 2150])
# )

## Figure 3: sensitivity: total net negative emissions

In [ ]:
selection_fig3 = data.select(selection, tcre=data.DEFAULT_TCRE, perc_reversible=1, endyear=2100, budget="600 GtCO2")

In [ ]:
import utils.fig3

In [ ]:
fig3 = utils.fig3.create_fig(selection_fig3)
fig3

In [ ]:
%%time
fig_SI_costs_sensitivity = utils.fig3.create_fig_costs(selection_fig3, 0.04)
fig_SI_costs_sensitivity

In [ ]:
# fig_SI_costs_sensitvity.write_image("../Paper/img/fig_SI_costs_sensitivity.png", scale=3)

## Figure 4: irreversible climate damages

In [ ]:
selection_damage_target = data.to_long(data.read_all("data/damagetarget/"))

In [ ]:
selection_damage_target_fig4 = data.select(selection_damage_target, with_overshoot=True, damage_i=[0, 2, -1], endyear=2100, tcre=data.DEFAULT_TCRE)

In [ ]:
import utils.fig4

In [ ]:
fig4 = utils.fig4.create_fig(selection_damage_target_fig4)
fig4

In [ ]:
# fig1ab.write_image("../Paper/img/fig1ab.png", scale=3)
# fig1c.write_image("../Paper/img/fig1c.png", scale=3)
# fig2.write_image("../Paper/img/fig2.png", scale=3)
# fig3.write_image("../Paper/img/fig3.png", scale=3)
# fig4.write_image("../Paper/img/fig4.png", scale=3)

# SI figures

## Carbon price and emissions for fig 2

In [ ]:
# variables = ['global_emissions']
def carbonprice_fig(selection):

    variables = 'carbonprice'
    selection_fig2_copy = selection.copy()
    selection_fig2_copy['carbonprice'] *= 1000
    selection_fig2_copy.loc[selection_fig2_copy['damage_coeff'] == data.DAMAGE_COEFFS[0], 'damage_coeff'] = 'DICE'
    selection_fig2_copy.loc[selection_fig2_copy['damage_coeff'] == data.DAMAGE_COEFFS[2], 'damage_coeff'] = 'Howard Total'
    selection_fig2_copy.loc[selection_fig2_copy['damage_coeff'] == data.DAMAGE_COEFFS[-1], 'damage_coeff'] = 'Burke (LR)'

    return px.line(
        selection_fig2_copy,
        x='year', y=variables,
        facet_col='damage_coeff',
        color='PRTP', line_dash='minlevel', color_discrete_sequence=plotparams.COLORS_PBL,
        render_mode='svg',
        width=950, height=500
    ).update_yaxes(col=1, title='Carbon price (USD / tCO<sub>2</sub>)')
fig_SI_carbonprices = carbonprice_fig(selection_fig2)
fig_SI_carbonprices

In [ ]:
# fig_SI_carbonprices.write_image("../Paper/img/fig_SI_carbonprices_600.png", scale=3)

## Carbon budget for 2°C

In [ ]:
selection_fig2_2C = data.select(selection, cost_level='p50', tcre=data.DEFAULT_TCRE, endyear=2100, damage_i=[0, 2, -1], perc_reversible=1, budget="1344 GtCO2")
fig2_2C = utils.fig2.create_fig(selection_fig2_2C, sdr=0.04, with_arrows=False)
# fig2_2C.write_image("../Paper/img/fig2_1344.png", scale=3)

In [ ]:
selection_fig3_2C = data.select(selection, tcre=data.DEFAULT_TCRE, perc_reversible=1, endyear=2100, budget="1344 GtCO2")
utils.fig3.create_fig(selection_fig3_2C).show()
utils.fig3.create_fig_costs(selection_fig3_2C, 0.04).update_layout(legend_y=0.8).show()

In [ ]:
fig_SI_carbonprices_2C = carbonprice_fig(selection_fig2_2C)
# fig_SI_carbonprices_2C.write_image("../Paper/img/fig_SI_carbonprices_1344.png", scale=3)